In [1]:
from bs4 import BeautifulSoup
# import http.client
import requests
import pandas as pd
import numpy as np

In [2]:
def data_scrape(year: int):
    url = "https://www.pro-football-reference.com/years/" + str(year)+"/passing.htm"
    # Reading the table found in the selected season
    html = pd.read_html(url, header=0)
    passing_df = html[0]
    clean_data = passing_df.drop(passing_df[passing_df.Age == 'Age'].index)
    clean_data = clean_data.fillna(0)
    passing_players = clean_data.drop(["Rk"], axis=1)

    # Replacing the position 0 to No Pos
    passing_players.loc[passing_players['Pos'] == 0, ['Pos']] = "No Pos"

    passing_players.rename(columns={'Yds.1': 'Yds Lost'}, inplace=True)

    # Converting data to numerical values
    passing_players[["Age", "G", "GS", "Cmp", "Att", "Cmp%", "Yds", "TD", "TD%", "Int", "Int%", "1D", "Lng", "Y/A", "AY/A", "Y/C", "Y/G", "Rate", "QBR", "Sk", "Yds Lost", "Sk%", "NY/A", "ANY/A", "4QC", "GWD"]] = passing_players[[
        "Age", "G", "GS", "Cmp", "Att", "Cmp%", "Yds", "TD", "TD%", "Int", "Int%", "1D", "Lng", "Y/A", "AY/A", "Y/C", "Y/G", "Rate", "QBR", "Sk", "Yds Lost", "Sk%", "NY/A", "ANY/A", "4QC", "GWD"]].apply(pd.to_numeric)

    passing_players.reset_index(drop=True, inplace=True)

    passing_players["Player"] = passing_players["Player"].map(lambda x: x.rstrip('*+'))

    return passing_players


In [3]:
def rush_data_scrape(year: int):
    # Define the URL that's going to be used to search for the data.
    url = "https://www.pro-football-reference.com/years/" + \
        str(year)+"/rushing.htm"
    # Set the reading for the table found in the site established before.
    html = pd.read_html(url, header=1)
    rushing_df = html[0]
    clean_data = rushing_df.drop(rushing_df[rushing_df.Age == 'Age'].index)
    clean_data = clean_data.fillna(0)
    rushing_players = clean_data.drop(['Rk'], axis=1)
    rushing_players[["Age", "G", "GS", "Att", "Yds", "TD", "1D", "Lng", "Y/A", "Y/G", "Fmb"]] = rushing_players[[
        "Age", "G", "GS", "Att", "Yds", "TD", "1D", "Lng", "Y/A", "Y/G", "Fmb"]].apply(pd.to_numeric)

    rushing_players.reset_index(drop=True, inplace=True)

    rushing_players.loc[rushing_players['Pos'] == 0, ['Pos']] = "No Pos"

    rushing_players["Player"] = rushing_players["Player"].map(
        lambda x: x.rstrip('*+'))

    return rushing_players


In [4]:
def league_avg(player_df: pd.DataFrame):
    temp_df = pd.DataFrame(columns=player_df.columns)
    temp_series = player_df.mean()

    temp_df = temp_df.append(temp_series, ignore_index=True)
    temp_df.drop(labels=["Player", "Tm", "Pos", "QBrec"], inplace=True, axis=1)
    return temp_df


In [5]:
def team_avg(player_df, team):
    temp_df = pd.DataFrame(columns=player_df.columns)

    temp_tm_df = pd.DataFrame(columns=player_df.columns)
    temp_tm_df = player_df.loc[player_df['Tm'] == team]
    temp_series = temp_tm_df.mean()

    temp_df = temp_df.append(temp_series, ignore_index=True)
    temp_df.drop(labels=["Player", "Tm", "Pos", "QBrec"], inplace=True, axis=1)

    return temp_df


In [6]:
def player_information(player_df, player_name):
    temp_df = pd.DataFrame(columns=player_df.columns)

    temp_tm_df = pd.DataFrame(columns=player_df.columns)
    temp_tm_df = player_df.loc[player_df["Player"] == player_name]

    temp_df = temp_df.append(temp_tm_df, ignore_index=True)
    temp_df.drop(labels=["Player", "Tm", "Pos"], inplace=True, axis=1)

    return temp_df


In [7]:
def check_extra_data(player_name):
    conn = http.client.HTTPSConnection("www.nfl.com")
    payload = ''
    headers = {}
    conn.request("GET", "/players/active/all?query=J.J.%20Watt", payload, headers)
    res = conn.getresponse()
    data = res.read()
    print(data.decode("utf-8"))
    return data


In [8]:
def additional_player_info(player_name):
    url = f"https://www.nfl.com/players/active/all?query={player_name}"
    page = requests.get(url)

    # Converting the HTML into Beautiful Soup
    soup = BeautifulSoup(page.text, 'html.parser')
    # Get the table's header row.
    header = soup.find('th', attrs={'scope': 'col'})
    # Extract the column names from Header Row
    columns = [col.get_text() for col in header.find_all('td')]

    return columns
    

In [9]:
def test_with_pandas(player_name):
    fixed_name = player_name.replace(" ","%20")
    url = f"https://www.nfl.com/players/active/all?query={fixed_name}"
    test_df = pd.read_html(
        url,
        attrs={"class":"d3-o-table d3-o-table--row-striping d3-o-table--detailed d3-o-team-stats--detailed"},
        header=0,
        index_col=1
    )[0]
    test_df.reset_index(inplace=True)
    return test_df

In [10]:
def team_name(player_df, filter_df):
    
    return 

In [11]:
passing_stats = data_scrape(2022)
passing_stats

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds Lost,Sk%,NY/A,ANY/A,4QC,GWD
0,Patrick Mahomes,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.4,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.6,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.8,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.5,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Christian Kirk,JAX,26,WR,17,17,0,0,1,0.0,...,0.0,39.6,10.9,0,0,0.0,0.00,0.00,0,0
102,Cooper Kupp,LAR,29,WR,9,9,0,0,1,0.0,...,0.0,39.6,2.8,0,0,0.0,0.00,0.00,0,0
103,James Proche,BAL,26,WR,15,0,0,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,0,0
104,Tommy Townsend,KAN,26,P,17,0,0,0,1,0.0,...,0.0,39.6,0.0,0,0,0.0,0.00,0.00,0,0


In [12]:
test_df = test_with_pandas(player_name="Christian McCaffrey")
test_df

,Current Team,Player,Position,Status
0,Carolina Panthers,Christian McCaffrey,RB,ACT


In [13]:
rushing_stats = rush_data_scrape(2022)
rushing_stats

,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
0,Derrick Henry,TEN,28,RB,16,16,349,1538,13,65,56,4.4,96.1,6
1,Josh Jacobs,LVR,24,RB,17,17,340,1653,12,93,86,4.9,97.2,3
2,Nick Chubb,CLE,27,RB,17,17,302,1525,12,69,41,5.0,89.7,1
3,Saquon Barkley,NYG,25,RB,16,16,295,1312,10,62,68,4.4,82.0,1
4,Najee Harris,PIT,24,RB,17,17,272,1038,7,45,36,3.8,61.1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,Adam Thielen,MIN,32,WR,17,17,1,4,0,0,4,4.0,0.2,0
362,Marquez Valdes-Scantling,KAN,28,WR,17,11,1,-3,0,0,-3,-3.0,-0.2,0
363,Jameson Williams,DET,21,WR,6,0,1,40,0,1,40,40.0,6.7,0
364,Cedrick Wilson Jr.,MIA,27,WR,15,0,1,8,0,0,8,8.0,0.5,0


## Testing to change _#TM_ to the latest team a player belonged to

In [14]:
tm_players = passing_stats.loc[passing_stats["Tm"].str.contains("TM", case=False)]
tm_players

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds Lost,Sk%,NY/A,ANY/A,4QC,GWD
28,Baker Mayfield,2TM,27,No Pos,12,10,2-8-0,201,335,60.0,...,180.3,79.0,24.8,36,217,9.7,5.25,4.81,0,0
77,Christian McCaffrey,2TM,26,RB,17,16,0,1,1,100.0,...,2.0,158.3,100.0,0,0,0.0,34.00,54.00,0,0
94,Chase Claypool,2TM,24,WR,15,11,0,1,1,100.0,...,0.1,118.7,100.0,0,0,0.0,1.00,21.00,0,0


In [15]:
tm_players_rush = rushing_stats.loc[rushing_stats["Tm"].str.contains("TM", case=False)]
tm_players_rush.sort_values(by="Tm", axis=0, ascending=False)

,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
59,Eno Benjamin,3TM,23,No Pos,15,3,77,313,2,14,45,4.1,20.9,0
8,Christian McCaffrey,2TM,26,RB,17,16,244,1139,8,59,49,4.7,67.0,1
172,Tony Jones,2TM,25,RB,6,0,10,24,0,3,7,2.4,4.0,0
346,Andy Isabella,2TM,26,No Pos,5,0,1,1,0,0,1,1.0,0.2,0
329,Jaelon Darden,2TM,23,No Pos,14,0,1,2,0,0,2,2.0,0.1,0
310,Amari Rodgers,2TM,23,No Pos,16,1,2,7,0,0,4,3.5,0.4,6
233,Jonathan Ward,2TM,25,No Pos,8,0,5,25,0,2,12,5.0,3.1,0
208,Kadarius Toney,2TM,23,WR,9,4,7,82,1,5,32,11.7,9.1,1
183,Chase Claypool,2TM,24,WR,15,11,9,59,0,3,15,6.6,3.9,1
151,Marlon Mack,2TM,26,No Pos,8,0,16,84,1,4,17,5.3,10.5,0


In [16]:
players_no_pos = rushing_stats.loc[rushing_stats["Pos"] == "No Pos"]
players_no_pos

,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
59,Eno Benjamin,3TM,23,No Pos,15,3,77,313,2,14,45,4.1,20.9,0
108,Baker Mayfield,2TM,27,No Pos,12,10,31,89,1,7,17,2.9,7.4,9
151,Marlon Mack,2TM,26,No Pos,8,0,16,84,1,4,17,5.3,10.5,0
233,Jonathan Ward,2TM,25,No Pos,8,0,5,25,0,2,12,5.0,3.1,0
310,Amari Rodgers,2TM,23,No Pos,16,1,2,7,0,0,4,3.5,0.4,6
329,Jaelon Darden,2TM,23,No Pos,14,0,1,2,0,0,2,2.0,0.1,0
346,Andy Isabella,2TM,26,No Pos,5,0,1,1,0,0,1,1.0,0.2,0
358,Ihmir Smith-Marsette,2TM,23,No Pos,8,0,1,-1,0,0,0,-1.0,-0.1,1


In [17]:
for i in tm_players["Player"]:
    print(i)

Baker Mayfield
Christian McCaffrey
Chase Claypool


In [18]:
type("Hello")

str